In [51]:
import json
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine

In [16]:
def get_sqlite_engine():
    engine = create_engine('sqlite:///data/reddit.db', echo=True)
    return engine

In [32]:
query = "SELECT day, blob FROM tickers_timeseries WHERE source='post';" 

In [33]:
engine = get_sqlite_engine()
with engine.begin() as con:
    raw_data = con.execute(query).fetchall()

2021-06-13 20:39:38,834 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-13 20:39:38,835 INFO sqlalchemy.engine.Engine SELECT day, blob FROM tickers_timeseries WHERE source='post';
2021-06-13 20:39:38,835 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-13 20:39:38,837 INFO sqlalchemy.engine.Engine COMMIT


In [35]:
# Extract unique tickers
unique_tickers = []
for values in raw_data:
    day, blob = values[0], json.loads(values[1])
    unique_tickers = unique_tickers + list(blob.keys())

In [48]:
# Making 2D datastructure
# [{'day':'2021-06-10', 'BB', '1'},
#  {'day':'2021-06-11', 'GME', '1'}, ..]

data = []
for values in raw_data:
    day, blob = values[0], json.loads(values[1])
    for ticker in blob:
        t_data = yf.Ticker(ticker)
#         if "shortName" not in t_data.info.keys():
#             continue
#         t_data.info
        data.append({
            'day':day,
            'ticker':ticker,
            'count':blob[ticker]
        })

In [53]:
df = pd.DataFrame(data)

In [54]:
df[df['ticker'] == 'GME']

,day,ticker,count
1,2021-06-10,GME,1
3,2021-06-11,GME,11
310,2021-06-13,GME,1
350,2021-06-14,GME,3


In [ ]:
# TODO: Graph timeseries